# Split tifs

## Inputs
This program takes the DEM_Slope.tif files and splits them into 1024x768 tifs and jpgs for use in training the model and running predictions.

DEM_Slope.tif files represent a PA State Game Land area and have an area number. The variable area_str represents the number in the program.

Also, each tif is either a PAN or PAS depending on the coordinate reference system (CRS) used for it. PAN is CRS 32128. PAS is 32129. Some Game Land areas have both PAN and PAS CRS .tifs, such as area 58. The variable panorpas represents whether the area tif file being processed is a PAN or a PAS.

To classify files for model training purposes, the program uses a shapefile of known charcoal hearths.
/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp

## Function
The program takes a large tif and then divides it into rectangles of standard size 1024x768 pixels. 

## Output
The program stores .tifs in:
/storage/images/charcoal_hearth_hill/images/split_tifs/

.jpgs in:
/storage/images/charcoal_hearth_hill/images/jpgs/

The program creates a .shp file to show the layout of the split tifs. These .shp files are in: 
/storage/images/charcoal_hearth_hill/polys/
Files are named: area_str + panorpas "_tile_boundary.shp"

In [64]:
# %%pycodestyle
# Run this to make sure all of the directories exist
data_directory = "/home/student/charcoal_hearth_hill/"
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
print(data_directory)
data_sub_directory = os.path.join(data_directory, "images")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)
data_sub_sub_directory = os.path.join(data_sub_directory, "jpgs")
if not os.path.exists(data_sub_sub_directory):
    os.makedirs(data_sub_sub_directory)
print(data_sub_sub_directory)
data_sub_sub_directory = os.path.join(data_sub_directory, "split_tifs")
if not os.path.exists(data_sub_sub_directory):
    os.makedirs(data_sub_sub_directory)
print(data_sub_sub_directory)
data_sub_directory = os.path.join(data_directory, "polys")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)
data_sub_directory = os.path.join(data_directory, "area_hearths")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)
data_sub_directory = os.path.join(data_directory, "annots")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)
data_sub_directory = os.path.join(data_directory, "images_training")
if not os.path.exists(data_sub_directory):
    os.makedirs(data_sub_directory)
print(data_sub_directory)

/home/student/charcoal_hearth_hill/
/home/student/charcoal_hearth_hill/images
/home/student/charcoal_hearth_hill/images/jpgs
/home/student/charcoal_hearth_hill/images/split_tifs
/home/student/charcoal_hearth_hill/polys
/home/student/charcoal_hearth_hill/area_hearths
/home/student/charcoal_hearth_hill/annots
/home/student/charcoal_hearth_hill/images_training


/home/student/charcoal_hearths_data/images/split_tifs/joe.tif


In [65]:
import geopandas as gpd
#https://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
from osgeo import osr, gdal

def get_poly_with_lat_long_from_geotif(geotif_fp):
    # Import necessary modules


    # get the existing coordinate system
    ds = gdal.Open(geotif_fp)

    old_cs= osr.SpatialReference()

    old_cs.ImportFromWkt(ds.GetProjectionRef())

    # create the new coordinate system
    nad83_wkt = """
    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4269"]]"""

    new_cs = osr.SpatialReference()

    new_cs .ImportFromWkt(nad83_wkt)

    # create a transform object to convert between coordinate systems
    transform = osr.CoordinateTransformation(old_cs,new_cs) 

    #get the point to transform, pixel (0,0) in this case
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3]

    #get the coordinates in lat long
    latlong1 = transform.TransformPoint(minx,miny)
    #print(latlong1)
    latlong2 = transform.TransformPoint(minx,maxy)
    #print(latlong2)
    latlong3 = transform.TransformPoint(maxx,maxy)
    #print(latlong3)
    latlong4 = transform.TransformPoint(maxx,miny)
    #print(latlong4)
    
    # Create an empty geopandas GeoDataFrame
    polydata = gpd.GeoDataFrame()
    polydata['geometry'] = None

    # Create a Polygon
    #print(latlong1[0], latlong1[1]), (latlong2[0], latlong2[1]), (latlong3[0], latlong3[1]), (latlong4[0], latlong4[1])
    coords = [(latlong1[0], latlong1[1]), (latlong2[0], latlong2[1]), (latlong3[0], latlong3[1]), (latlong4[0], latlong4[1])]
    poly = Polygon(coords)

    return(poly)

import numpy as np

def tif_to_jpg(tif_file,output_file_name, output_folder):
    #print(tif_file,output_file_name, output_folder)
    ds = gdal.Open(tif_file)
    #geoTrans = srcImage.GetGeoTransform()

    band = ds.GetRasterBand(1)
    width = ds.RasterXSize
    height = ds.RasterYSize

    data = band.ReadAsArray(0, 0, width, height)
    #convert all the bad data
    data[data==-9999.0] = 0
    max_value = numpy.max(data)
    color_multiplier = 255/(max_value-1)
    #print(color_multiplier)
    data = data*color_multiplier
    #print(data)
    data_int = np.array(data, dtype='int')
    #print(data_int)
    #clip = ds.readasarray(ds)
    data_int = data_int.astype(gdalnumeric.numpy.uint8)

    gdalnumeric.SaveArray(data_int, output_folder+output_file_name+".jpg", format="JPEG")

#https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html?highlight=rasterize

# Run this
def write_annot(obj_type, obj_annots_dir, obj_f_num, org_f_path,org_f_width,org_f_height,org_f_depth, refPts):
    # This generates the xml file annotation that is consumed by the MASK R-CNN process.
    # With credit to: # https://www.geeksforgeeks.org/reading-writing-text-files-python/
    annot_file_path = obj_annots_dir+obj_f_num+'.xml'
    annot_file = open(annot_file_path,"w") 

    annot_file.write("<annotation>\n") 
    annot_file.write("	<folder>"+obj_type+"</folder>\n") 
    annot_file.write("	<filename>"+obj_f_num+".jpg</filename>\n") 
    annot_file.write("	<path>"+org_f_path+"</path>\n") 
    annot_file.write("	<source>\n") 
    annot_file.write("		<database>Muhlenberg_charcoal_hearths</database>\n") 
    annot_file.write("	</source>\n") 
    annot_file.write("	<size>\n") 
    annot_file.write("		<width>"+str(org_f_width)+"</width>\n") 
    annot_file.write("		<height>"+str(org_f_height)+"</height>\n") 
    annot_file.write("		<depth>"+str(org_f_depth)+"</depth>\n") 
    annot_file.write("	</size>\n") 
    annot_file.write("	<segmented>0</segmented>\n") 
    for ocn in range(0,len(refPts)):
        refPt = refPts[ocn]
        refPtMin = refPt[0]
        refPtMax = refPt[1]
        
        #Sometimes the mouse gets dragged from the bottom to the top, etc
        if(refPtMin[1]>refPtMax[1]):
            ytemp = refPtMin[1]
            refPtMin[1] = refPtMax[1]
            refPtMax[1]=ytemp
        if(refPtMin[0]>refPtMax[0]):
            xtemp = refPtMin[0]
            refPtMin[0] = refPtMax[0]
            refPtMax[0]=xtemp 
        
        annot_file.write("	<object>\n") 
        annot_file.write("		<name>"+obj_type+"</name>\n") 
        annot_file.write("		<number>"+str(ocn)+"</number>\n") 
        annot_file.write("		<truncated>0</truncated>\n") 
        annot_file.write("		<difficult>0</difficult>\n") 
        annot_file.write("		<bndbox>\n") 
        annot_file.write("			<xmin>"+str(refPtMin[0])+"</xmin>\n") 
        annot_file.write("			<ymin>"+str(refPtMin[1])+"</ymin>\n") 
        annot_file.write("			<xmax>"+str(refPtMax[0])+"</xmax>\n") 
        annot_file.write("			<ymax>"+str(refPtMax[1])+"</ymax>\n") 
        annot_file.write("		</bndbox>\n") 
        annot_file.write("	</object>\n") 
    annot_file.write("</annotation>\n") 
    annot_file.close()

In [66]:
import gdal
from osgeo import ogr
def get_pixel_with_geot(geoT, point_x, point_y):
    """Get the pixel for given coordinates (in the raster's convention, not
    checked!) for a raster file.

    Parameters
    ----------
    raster: string
        A GDAL-friendly raster filename
    point_x: float
        The Easting in the same coordinates as the raster (not checked!)
    point_y: float
        The Northing in the same coordinates as the raster (not checked!)

    Returns
    -------
    The row/column (or column/row, depending on how you define it)
    """
    #g = gdal.Open(raster)
    #if g is None:
    #    raise ValueError(f"{raster:s} cannot be opened!")
    #geoT = open_raster.GetGeoTransform()
    inv_geoT = gdal.InvGeoTransform(geoT)
    r, c = (gdal.ApplyGeoTransform(inv_geoT, point_x, point_y))
    return int(r + 0.5), int(c + 0.5)

def annotate_tif_if_it_has_hearths(area_hearth_data, tif_poly, tif_fp, annot_file_name, geotif_crs):
    # Create an empty geopandas GeoDataFrame
    hearth_data = gpd.GeoDataFrame()
    hearth_data['geometry'] = None
    hearth_data.crs = {'init':'epsg:4326'}

    # Create an empty geopandas GeoDataFrame
    hearth_data_poly = gpd.GeoDataFrame()
    hearth_data_poly['geometry'] = None
    hearth_data_poly.crs = {'init':'epsg:4326'}

    selection = area_hearth_data[0:]
    point_count=0
    
    print(tif_fp)
    src_ds=gdal.Open(tif_fp) 
    gt=src_ds.GetGeoTransform()
    
    # create a transform object to convert between coordinate systems
    #transform = osr.CoordinateTransformation(old_cs,new_cs)
    
    target = osr.SpatialReference(wkt=src_ds.GetProjection())
    source = osr.SpatialReference()
    #source.ImportFromEPSG(4326)
    #source.ImportFromEPSG(32128)
    #target.ImportFromEPSG(32128)
    source.ImportFromEPSG(geotif_crs)
    target.ImportFromEPSG(geotif_crs)
    
    transform = osr.CoordinateTransformation(source, target)

    for index, row in selection.iterrows():
        #print("for index, row in selection.iterrows():")
        pt = row['geometry']
        #print(row)
        #select only points inside the bounds of the geotiff AND are confirmed to be hearths
        if(pt.within(tif_poly)==True):
            #print("if(pt.within(tif_poly)==True):")
            point_count=point_count+1
            #print(pt, pt.within(tif_poly))
            hearth_data = hearth_data.append(row, ignore_index=True)
            
            polygon_half_size = 0.0000895078
            polygon_half_size = 0.0001745402
            #get the coordinates in lat long
            # pp_ = point_poly - the polygon around the points
            pp_minx=pt.bounds[0]-polygon_half_size
            #print(pt.bounds[0])
            pp_maxx=pt.bounds[0]+polygon_half_size
            pp_miny=pt.bounds[1]-polygon_half_size
            pp_maxy=pt.bounds[1]+polygon_half_size
            
            pp_latlong1 = transform.TransformPoint(pp_minx,pp_miny)
            #print("latlong1",latlong1)
            print("pp_minx",pp_minx)
            pp_latlong2 = transform.TransformPoint(pp_minx,pp_maxy)
            #print(latlong2)
            pp_latlong3 = transform.TransformPoint(pp_maxx,pp_maxy)
            #print(latlong3)
            pp_latlong4 = transform.TransformPoint(pp_maxx,pp_miny)
            #print(latlong4)
            # Create a Polygon
            pp_coords = [(pp_minx, pp_miny), (pp_minx, pp_maxy), (pp_maxx, pp_maxy), (pp_maxx, pp_miny)]
            ppoly = Polygon(pp_coords)
            new_pp_row = {'id':row['id'], 'geometry':ppoly}
            #hearth_data_poly.loc[0, 'geometry'] = ppoly
            #hearth_data_poly.loc[0, 'id'] = row['id']
            hearth_data_poly = hearth_data_poly.append(new_pp_row, ignore_index=True)
    #Do we have points inside this tif?  If so, annotate and copy>0
    if(point_count)>0:

        #hearth_data_poly = hearth_data_poly.to_crs(epsg=32128)
        #hearth_data = hearth_data.to_crs(epsg=32128)
        hearth_data_poly = hearth_data_poly.to_crs(epsg=geotif_crs)
        hearth_data = hearth_data.to_crs(epsg=geotif_crs)
    
        #Write out shp files for diagnostics
        outfp_base = tif_fp[:-4]
        outfp = outfp_base+"_hearth_data_poly.shp"
        # Write the data into that Shapefile
        hearth_data_poly.to_file(outfp)
        outfp = outfp_base+"_hearth_data.shp"
        # Write the data into that Shapefile
        hearth_data.to_file(outfp)
        print("****")

        #Get the dimentions of column and row
        nc   = src_ds.RasterXSize
        nr   = src_ds.RasterYSize
        print("src_ds.RasterXSize", nc,nr)
        gdal.UseExceptions() #so it doesn't print to screen everytime point is outside grid
        
        #target = osr.SpatialReference(wkt=src_ds.GetProjection())
        #source = osr.SpatialReference()
        #source.ImportFromEPSG(4326)
        #transform = osr.CoordinateTransformation(source, target)
        refPts = []      
        refPt = []
        #I'm hoping the tifs will be 1024X768        
        # Make annotation file for the jpeg from polygons
        for index, row in hearth_data_poly.iterrows():
            poly = row['geometry']
            #print(row, row['id'])
            #print(poly)
            
            boundary = poly.bounds
            print("boundary = poly.bounds")
            print(boundary[0])
            print(boundary[1])
            print(boundary[2])
            print(boundary[3])

            lon = boundary[0]
            lat = boundary[1]
    
            print("lon,lat",lon, lat)
      
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            point.Transform(transform)

            #p1x,p1y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
            p1x,p1y = get_pixel_with_geot(gt, point.GetX(), point.GetY())
            print("p1",point.GetX(), point.GetY(),p1x,p1y)
        
            #cx,cy=convert_coordinates(lon, lat)
            #p1x,p1y= (get_pixel_with_geot(gt,cx,cy ))
            lon = boundary[2]
            lat = boundary[3]
    
            print("lon,lat",lon, lat)
        
            #cx,cy=convert_coordinates(lon, lat)
            #p2x,p2y= (get_pixel_with_geot(gt,cx,cy ))
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            point.Transform(transform)

            #p2x,p2y = world_to_pixel(ds.GetGeoTransform(), point.GetX(), point.GetY())
            p2x,p2y = get_pixel_with_geot(gt, point.GetX(), point.GetY())
            print("p2",point.GetX(), point.GetY(),p2x,p2y)

            #p2y is smaller than p1y so we'll exchange them to keep p1 as the minimum.
            ptemp=p1y
            p1y=p2y
            p2y=ptemp
            if(p1x<0):
                p1x=0
            if(p1y<0):
                p1y=0
            if(p2x<0):
                p2x=0
            if(p2y<0):
                p2y=0

            if(p1x>nc):
                p1x=nc
            if(p1y>nr):
                p1y=nr
            if(p2x>nc):
                p2x=nc
            if(p2y>nr):
                p2y=nr
     
        
            refPt = [(p1x, p1y)]
            refPt.append((p2x, p2y))
            refPts.append(refPt)

        object_type = 'charcoal_hearth_hill'
        
        object_annots_dir = '/home/student/charcoal_hearth_hill/annots/'
        write_annot(object_type, object_annots_dir, annot_file_name, annot_file_name+".jpg",nc,nr,1, refPts)
        
        images_training_dir = '/home/student/charcoal_hearth_hill/images_training/'
        tif_to_jpg(tif_fp,annot_file_name,images_training_dir)
    

In [67]:
# Thanks to
#https://gis.stackexchange.com/questions/92207/split-a-large-geotiff-into-smaller-regions-with-python-and-gdal
import os  
import numpy
#import gdal
from osgeo import gdal, osr
import math
from itertools import chain
#import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import gdalnumeric

def get_extent(dataset):

    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    transform = dataset.GetGeoTransform()
    minx = transform[0]
    maxx = transform[0] + cols * transform[1] + rows * transform[2]

    miny = transform[3] + cols * transform[4] + rows * transform[5]
    maxy = transform[3]

    return {
            "minX": str(minx), "maxX": str(maxx),
            "minY": str(miny), "maxY": str(maxy),
            "cols": str(cols), "rows": str(rows)
            }

def create_tiles(minx, miny, maxx, maxy, nw, nh):
    width = maxx - minx
    height = maxy - miny

    matrix = []

    for j in range(nw, 0, -1):
        for i in range(0, nh):

            ulx = minx + (width/nw) * i # 10/5 * 1
            uly = miny + (height/nh) * j # 10/5 * 1

            lrx = minx + (width/nw) * (i + 1)
            lry = miny + (height/nh) * (j - 1)
            matrix.append([[ulx, uly], [lrx, lry]])

    return matrix



def create_standard_tiles(minx, miny, maxx, maxy):
    print("standard",minx, miny, maxx, maxy)
    width = maxx - minx
    height = maxy - miny
    
    standard_width = 1024
    standard_height = 768
   
    nw = int(width/standard_width)+1
    nh = int(height/standard_height)+1
    print(nw,nh)
    #print(nw*standard_width,nh*standard_height)
    
    matrix = []

    for j in range(nh, 0, -1):
        for i in range(0, nw):

            ulx = minx + (standard_width * i) # 10/5 * 1
            uly = miny + (standard_height * j) # 10/5 * 1

            lrx = minx + (standard_width * (i + 1))
            lry = miny + (standard_height * (j - 1))
            if(lrx>maxx):
                lrx=maxx
            if(lrx<minx):
                lrx=minx
              
            if(lry>maxy):
                lry=maxy
            if(lry<miny):
                lry=miny

            if(ulx>maxx):
                ulx=maxx
            if(ulx<minx):
                ulx=minx

            if(uly>maxy):
                uly=maxy
            if(uly<miny):
                uly=miny

            #print("matrix:",ulx, uly,lrx, lry)
            matrix.append([[ulx, uly], [lrx, lry],[i,j]])
    print("standard",minx, miny, maxx, maxy)
    return matrix


def split(file_name, geotif_crs):
    data_directory = "/home/student/charcoal_hearth_hill/"
    jpgs_output_folder = os.path.join(data_directory,"images/","jpgs/")
    polys_output_folder = os.path.join(data_directory,"polys/")

    #Some areas have both a PAN and a PAS, so we need to distinguish them
    panorpas=""
    if(geotif_crs==32129):
        panorpas="pas"
    if(geotif_crs==32128):
        panorpas="pan"

    if(os.path.exists(file_name)):
        from osgeo import osr
        raw_file_name = os.path.splitext(os.path.basename(file_name))[0].replace("_downsample", "")
        print(raw_file_name)
        area_num=raw_file_name[:3]
        if(area_num[-1:] =="b"):
            area_num=area_num[:-1]
        area_num = int(area_num)
        area_str = "0"+str(area_num)
        area_str = area_str[-3:]
        print(area_str)
        driver = gdal.GetDriverByName('GTiff')
        dataset = gdal.Open(file_name)
        proj_dataset = osr.SpatialReference(wkt=dataset.GetProjection())
        print(proj_dataset.GetAttrValue('AUTHORITY',1),geotif_crs)
        
        # Check that the geotif_crs matches what is in the file.
        if(proj_dataset.GetAttrValue('AUTHORITY',1)==str(geotif_crs)):
            print(proj_dataset)
   
            band = dataset.GetRasterBand(1)
            transform = dataset.GetGeoTransform()

            extent = get_extent(dataset)

            cols = int(extent["cols"])
            rows = int(extent["rows"])
            print ("Columns: ", cols)
            print ("Rows: ", rows)

            minx = float(extent["minX"])
            maxx = float(extent["maxX"])
            miny = float(extent["minY"])
            maxy = float(extent["maxY"])
        
            width = int(maxx - minx)
            height = int(maxy - miny)

            nw = int(width/1024)+1
            nh = int(height/768)+1
    
            #******
            # make a polygon for the area and see what points are in the whole area.
            # get the coordinates in lat long

            # Create an empty geopandas GeoDataFrame
            area_hearth_data = gpd.GeoDataFrame()
            area_hearth_data['geometry'] = None
            area_poly=get_poly_with_lat_long_from_geotif(file_name)
            # Read file using gpd.read_file()
            all_hearth_data = gpd.read_file("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp")
            selection = all_hearth_data[0:]
            #print(selection)
            for index, row in selection.iterrows():
                #print("for index, row in selection.iterrows():")
                pt = row['geometry']
                #print(row)
                #select only points inside the bounds of the geotiff AND are confirmed to be hearths
                if(pt.within(area_poly)==True):
                    #print("if(pt.within(tif_poly)==True):")
                    area_hearth_data = area_hearth_data.append(row, ignore_index=True)
    
            if(not area_hearth_data.empty):
            # Determine the output path for the Shapefile
                outfp = os.path.join(data_directory, "area_hearths", "area_" + area_str + panorpas + "_hearth.shp")
                # Write the data into that Shapefile
                area_hearth_data.to_file(outfp)
            #*************************
            #output_path = os.path.join("data", raw_file_name)
            #if not os.path.exists(output_path):
            #    os.makedirs(output_path)

            # Polygons to reflect the tiles
            # Create an empty geopandas GeoDataFrame
            tile_boundary_poly = gpd.GeoDataFrame()
            tile_boundary_poly['geometry'] = None
            tile_boundary_poly.crs = {'init':'epsg:4326'}

            #tiles = create_tiles(minx, miny, maxx, maxy, nw,nh)
            tiles = create_standard_tiles(minx, miny, maxx, maxy)
            transform = dataset.GetGeoTransform()
            xOrigin = transform[0]
            yOrigin = transform[3]
            pixelWidth = transform[1]
            pixelHeight = -transform[5]

            srs = osr.SpatialReference()
            #srs.ImportFromEPSG(32128)
            srs.ImportFromEPSG(geotif_crs)
            #srs.ImportFromEPSG(4326)
            #dst_ds.SetProjection( srs.ExportToWkt() )
    
            tile_num = 0
            for tile in tiles:
                #print (tile)

                minx = tile[0][0]
                maxx = tile[1][0]
                miny = tile[1][1]
                maxy = tile[0][1]
                tilex = "00"+str(tile[2][0])
                tilex = tilex[-2:]
                tiley = "00"+str(tile[2][1])
                tiley = tiley[-2:]
        
                p1 = (minx, maxy)
                p2 = (maxx, miny)

                i1 = int((p1[0] - xOrigin) / pixelWidth)
                j1 = int((yOrigin - p1[1]) / pixelHeight)
                i2 = int((p2[0] - xOrigin) / pixelWidth)
                j2 = int((yOrigin - p2[1]) / pixelHeight)

                new_cols = i2-i1
                new_rows = j2-j1

                data = band.ReadAsArray(i1, j1, new_cols, new_rows)
                #print("data",data.mean(),data)
                #if len(list(set(chain(*data))))<3:
                if(np.all(data==data[0]) or (np.amin(data)==-9999 and np.amax(data)==0)):
                    print("np.all(data==data[0])")
                    #print("data.mean",data.mean())
                else:
                    print("data.mean.too",data.mean(),np.amin(data),np.amax(data) )
                
                    new_x = xOrigin + i1*pixelWidth
                    new_y = yOrigin - j1*pixelHeight

                    #print (new_x, new_y)

                    new_transform = (new_x, transform[1], transform[2], new_y, transform[4], transform[5])
                    output_file_name = area_str + panorpas + tiley + tilex
                    output_file_base = output_file_name + ".tif"
                    output_file = os.path.join(data_directory,"images","split_tifs",output_file_base)

                    dst_ds = driver.Create(output_file,
                                           new_cols,
                                           new_rows,
                                           1,
                                           gdal.GDT_Float32)

                    #writing output raster
                    dst_ds.GetRasterBand(1).WriteArray( data )
                    dst_ds.SetProjection( srs.ExportToWkt() )
                    dst_ds.SetGeoTransform(new_transform)
                    tif_metadata = {
                        "minX": str(minx), "maxX": str(maxx),
                        "minY": str(miny), "maxY": str(maxy)
                    }
                    dst_ds.SetMetadata(tif_metadata)
                    dst_ds = None            

                    #make a polygon for this tile
                    tpoly = get_poly_with_lat_long_from_geotif(output_file)
                    new_tp_row = {'id':tile_num, 'geometry':tpoly}
                    tile_boundary_poly = tile_boundary_poly.append(new_tp_row, ignore_index=True)
            
                    # Save .jpg
                    # close tif before re-opening it and saving as jpg
                    # The jpg is to be consumed by the model.
                    
                    tif_to_jpg(output_file,output_file_name,jpgs_output_folder)
                    # /storage/images/charcoal_hearth_hill/images_training/
                    # Are there points inside the geotiff?
                    # annotate_tif_if_it_has_hearths("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp",tpoly,output_file,output_file_name)
                    annotate_tif_if_it_has_hearths(area_hearth_data,tpoly,output_file,output_file_name,geotif_crs)
                
                    #We need to refine to handle points on the border
            
                #Close output raster dataset
                dst_ds = None

                tile_num += 1

            dataset = None
    
            # Determine the output path for the Shapefile
            tile_boundary_poly = tile_boundary_poly.to_crs(epsg=geotif_crs)
    
            outfp = os.path.join(polys_output_folder, (area_str + panorpas + "_tile_boundary.shp"))
            # Write the data into that Shapefile
            tile_boundary_poly.to_file(outfp)
        else:
            print(proj_dataset.GetAttrValue('AUTHORITY',1),geotif_crs," don't match.")
    else:
        print(file_name+" does not exist.")


# Instructions
Download the Slope_DEM.tif files
PAN = 32128
PAS = 32129
Run all cells above and then the one below.

In [ ]:
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/39/39blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/41/41blast2demPAS_Slope_DEM.tif",32129)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/43/43blast2demPAS_Slope_DEM.tif",32129)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/46/46blast2demPAS_Slope_DEM.tif",32129)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/52/52partialDEM_Slope.tif",26918)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/55/55blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Lidar_and_Raster_Images_5-20-2020/58/58blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/12blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/13blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/14blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/24blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/25blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/26blast2demPAN_Slope_DEM.tif",32128) - AttributeError: 'NoneType' object has no attribute 'GetProjection'
#split("/storage/images/28blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/29blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/30blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/31blast2demPAN_Slope_DEM.tif",32128)
#33 has PAN and PAS
#split("/storage/images/part_1/33blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/33blast2demPAS_Slope_DEM.tif",32129)

#split("/storage/images/part_1/34blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/35blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/36blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/37blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/38blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/39blast2demPAN_Slope_DEM.tif",32128)
#split("/storage/images/part_1/40blast2demPAN_Slope_DEM.tif",32128)



#split("/storage/images/part_4/320blast2demPAN_DEM_Slope.tif",32128)

#52partialDEM_Slope.tif",26918)
#58 has PAN and PAS
#60 has PAN and PAS
#83 is missing - now there.
#84 has PAN and PAS
#95 has PAN and PAS
#105 has PAN and PAS
#108 has PAN and PAS
#109 is missing

# on /storage
# 12:29:18 Current Time = 12:30:02 == 44 secs

#on home
# 12:31:29 12:31:51 == 22 secs
        
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

split("/storage/images/part_1/12blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/13blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/14blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/24blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/25blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/26blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/28blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/29blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/30blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/31blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/33blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/33blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/34blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/35blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/36blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/37blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/38blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/39blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/40blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/41blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/42blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/43blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/44blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/45blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/46blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/47blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/48blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/49blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/50blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/51blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/52blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/53blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/54blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/55blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/56blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/57blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/58blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/58blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/59blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/60blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/60blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/61blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/62blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/63blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/64blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/65blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/66blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/67blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/68blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/69blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/70blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/71blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/72blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/73blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/74blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/75blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/76blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/77blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/78blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/79blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/80blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/81blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/82blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/83blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/84blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/84blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/85blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/86blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/87blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/88blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/89blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/90blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/91blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/92blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/93blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/94blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/95blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/95blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_1/96blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_1/97blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/100blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/101blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/102blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/103blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/104blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/105blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/105blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/106blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/107blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/108blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/108blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/109blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/110blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/111blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/112blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/113blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/114blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/115blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/116blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/117blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/118blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/119blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/120blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/120blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/121blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/122blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/123blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/124blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/126blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/127blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/128blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/129blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/130blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/131blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/132blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/132blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/133blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/134blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/135blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/136blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/137blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/137blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/138blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/139blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/140blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/141blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/142blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/143blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/144blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/145blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/146blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/147blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/148blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/149blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/150blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/151blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/151blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/152blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/153blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/154blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/155blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/156blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/157blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/158blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/158blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/159blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/160blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/161blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/162blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/163blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/164blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/165blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/166blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/167blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/168blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/168blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/169blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/170blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/171blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/172blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/173blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_2/98blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_2/99blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/174blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/175blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/176blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/178blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/179blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/180blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/181blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/182blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/183blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/184blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/185blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/186blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/187blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/188blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/189blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/190blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/191blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/192blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/193blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/193blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/194blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/195blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/196blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/197blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/198blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/199blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/200blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/201blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/201blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/202blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/203blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/204blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/205blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/206blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/207blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/208blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/209blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/210blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/211blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/212blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/212blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/213blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/214blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/215blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/216blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/217blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/217blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/218blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/219blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/220blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/221blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/222blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/223blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/224blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/225blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/226blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/227blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/228blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/229blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/230blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/231blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/232blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/233blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/233blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/234blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/235blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/236blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/237blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/238blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/239blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/242blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/243blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/244blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/245blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/246blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/247blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/248blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/249blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_3/250blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_3/251blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/252blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/253blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/254blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/255blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/256blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/257blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/257blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/258blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/258blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/259blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/260blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/261blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/262blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/263blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/264blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/265blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/266blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/267blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/268blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/269blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/270blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/271blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/272blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/273blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/274blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/275blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/276blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/277blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/278blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/278blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/279blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/280blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/281blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/282blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/283blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/284blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/284blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/285blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/286blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/287blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/288blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/290blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/291blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/292blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/293blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/294blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/295blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/296blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/297blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/298blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/299blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/300blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/301blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/302blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/303blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/304blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/305blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/306blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/307blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/309blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/310blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/311blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/312blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/313blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/314blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/315blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/316blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/317blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/317blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/318blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/320blast2demPAN_DEM_Slope.tif",32128)
split("/storage/images/part_4/321blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/322blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/323blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/324blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/325blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/326blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/327blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/328blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/329blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/329blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/330blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/331blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/332blast2demPAS_Slope_DEM.tif",32129)
split("/storage/images/part_4/333blast2demPAN_Slope_DEM.tif",32128)
split("/storage/images/part_4/335blast2demPAN_Slope_DEM.tif",32128)

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)



In [ ]:
# QGIS only
# This is just here to store and keep track of.
# In QGIS due to loading a large number of files, it gets unstable.
# This program loads all of the area polys to make sure processing worked correctly.
# It will print which layers did not load.
from qgis.core import *
import qgis.utils

# get the path to the shapefile e.g. /home/project/data/ports.shp
path_to_layer = "/storage/images/charcoal_hearth_hill/downloads/Weston_Uploads/Shapefiles_5-20-2020/Charcoal-Hearths.shp"
#/storage/images/charcoal_hearth_hill/polys/
# The format is:
# vlayer = QgsVectorLayer(data_source, layer_name, provider_name)

   
vlayer = iface.addVectorLayer(path_to_layer, "Charcoal Hearths layer", "ogr")
if not vlayer:
    print("Layer failed to load!")

report=""
import csv
with open('/home/student/charcoalhearths/data_sheet.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row['Area'])
        area=row['Area']
        area="0"+area
        area=area[-3:]
        pan=""
        pas=""
        pan = row['PAN']
        if(pan == "Y"):
            path_to_layer = "/home/student/charcoal_hearth_hill/polys/"+area+"pan_tile_boundary.shp"
            vlayer = iface.addVectorLayer(path_to_layer, area+"pan layer", "ogr")
            if not vlayer:
                print(area+ "pan layer failed to load!")
                report = report + "\n" + area+ "pan layer failed to load!"
        pas = row['PAS']
        if(pas == "Y"):
            path_to_layer = "/home/student/charcoal_hearth_hill/polys/"+area+"pas_tile_boundary.shp"
            vlayer = iface.addVectorLayer(path_to_layer, area+"pas layer", "ogr")
            if 